In [ ]:
import pandas as pd

In [ ]:
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_val_score,LeavePGroupsOut, LeaveOneGroupOut, cross_val_predict
from sklearn.metrics import confusion_matrix, make_scorer, f1_score, accuracy_score, recall_score, precision_score
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel, RFECV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
pd.options.mode.chained_assignment = None

In [ ]:
filename = 'facies_PE_ET_A.csv'
training_data = pd.read_csv(filename)
print(set(training_data["Well Name"]))
training_data.head()

{'KIMZEY A', 'SHRIMPLIN', 'Recruit F9', 'ALEXANDER D', 'CROSS H CATTLE', 'CHURCHMAN BIBLE', 'SHANKLE', 'LUKE G U', 'NOLAN', 'NEWBY'}


,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,ILD log10/PHIND log10
0,3,A1 SH,SHRIMPLIN,2793.0,77.45,0.664,9.9,11.915,4.6,1,1.000,0.617046
1,3,A1 SH,SHRIMPLIN,2793.5,78.26,0.661,14.2,12.565,4.1,1,0.979,0.601367
2,3,A1 SH,SHRIMPLIN,2794.0,79.05,0.658,14.8,13.050,3.6,1,0.957,0.589812
3,3,A1 SH,SHRIMPLIN,2794.5,86.10,0.655,13.9,13.115,3.5,1,0.936,0.585989
4,3,A1 SH,SHRIMPLIN,2795.0,74.58,0.647,13.5,13.300,3.4,1,0.915,0.575699


In [ ]:
well_data = pd.read_csv('validation_data_nofacies.csv')
print(set(well_data["Well Name"]))
print(well_data.shape)
well_data.head()

{'CRAWFORD', 'STUART'}
(800, 12)


,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,PHIND_log10,ILD log10/PHIND log10
0,A1 SH,STUART,2808.0,66.276,0.630,3.3,10.65,3.591,1,1.000,1.027350,0.613228
1,A1 SH,STUART,2808.5,77.252,0.585,6.5,11.95,3.341,1,0.978,1.077368,0.542990
2,A1 SH,STUART,2809.0,82.899,0.566,9.4,13.60,3.064,1,0.956,1.133539,0.499321
3,A1 SH,STUART,2809.5,80.671,0.593,9.5,13.25,2.977,1,0.933,1.122216,0.528419
4,A1 SH,STUART,2810.0,75.971,0.638,8.7,12.35,3.020,1,0.911,1.091667,0.584427


In [ ]:
# concat train and test for processing 
well_data["origin"] = 'test'
training_data["origin"] = 'train'
df = pd.concat([well_data,training_data],axis=0,ignore_index=True)[list(training_data.columns)]
df['Well Name'] = df['Well Name'].astype('category')
df.head(10)

,Facies,Formation,Well Name,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS,ILD log10/PHIND log10,origin
0,NaN,A1 SH,STUART,2808.0,66.276,0.630,3.3,10.65,3.591,1,1.000,0.613228,test
1,NaN,A1 SH,STUART,2808.5,77.252,0.585,6.5,11.95,3.341,1,0.978,0.542990,test
2,NaN,A1 SH,STUART,2809.0,82.899,0.566,9.4,13.60,3.064,1,0.956,0.499321,test
3,NaN,A1 SH,STUART,2809.5,80.671,0.593,9.5,13.25,2.977,1,0.933,0.528419,test
4,NaN,A1 SH,STUART,2810.0,75.971,0.638,8.7,12.35,3.020,1,0.911,0.584427,test
5,NaN,A1 SH,STUART,2810.5,73.955,0.667,6.9,12.25,3.086,1,0.889,0.612975,test
6,NaN,A1 SH,STUART,2811.0,77.962,0.674,6.5,12.45,3.092,1,0.867,0.615430,test
7,NaN,A1 SH,STUART,2811.5,83.894,0.667,6.3,12.65,3.123,1,0.844,0.605213,test
8,NaN,A1 SH,STUART,2812.0,84.424,0.653,6.7,13.05,3.121,1,0.822,0.585330,test
9,NaN,A1 SH,STUART,2812.5,83.160,0.642,7.3,12.95,3.127,1,0.800,0.577198,test


In [ ]:
import math

In [ ]:

# nb points : can be correlated with how soft soil is ? 
print("session")
sessionsize = df.groupby(["Well Name",'Formation']).size().reset_index()
sessionsize.columns =  ["Well Name",'Formation','formation_size']
df = pd.merge(df,sessionsize,how='left',on = ["Well Name",'Formation'])

# depth : 
print("depth")
sessionsize = df.groupby(["Well Name",'Formation'])["Depth"].min().reset_index()
sessionsize.columns =  ["Well Name",'Formation','minimum_depth']
df = pd.merge(df,sessionsize,how='left',on = ["Well Name",'Formation'])

sessionsize = df.groupby(["Well Name",'Formation'])["Depth"].max().reset_index()
sessionsize.columns =  ["Well Name",'Formation','maximum_depth']
df = pd.merge(df,sessionsize,how='left',on = ["Well Name",'Formation'])


df["formation_depth"] = df["maximum_depth"] - df["minimum_depth"]
df["soft_indic"] = df["formation_depth"] / df["formation_size"]

session
depth


In [ ]:
df.to_csv('added features.csv')

NameError: ignored